<a href="https://colab.research.google.com/github/borundev/pytorch_examples/blob/master/ClassificationWithUncertainity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-cache-dir -Uqq git+https://github.com/borundev/pytorch_examples.git

In [2]:
!rm /usr/local/lib/python3.7/dist-packages/torch/optim/adamw.py
!wget https://raw.githubusercontent.com/pytorch/pytorch/master/torch/optim/adamw.py -P /usr/local/lib/python3.7/dist-packages/torch/optim/

--2021-05-06 10:17:19--  https://raw.githubusercontent.com/pytorch/pytorch/master/torch/optim/adamw.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4929 (4.8K) [text/plain]
Saving to: ‘/usr/local/lib/python3.7/dist-packages/torch/optim/adamw.py’

adamw.py            100%[===================>]   4.81K  --.-KB/s    in 0s      

2021-05-06 10:17:19 (81.3 MB/s) - ‘/usr/local/lib/python3.7/dist-packages/torch/optim/adamw.py’ saved [4929/4929]



In [3]:
!pip install -Uqq  names

In [4]:
import torch
torch.__version__

'1.8.1+cu101'

In [5]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb
import names
import pytorch_examples

In [6]:
from pytorch_examples.transfer_learning.model import CustomModel

In [7]:
from data.fudge_labels import modify_data_module
from data.cats_and_dogs import CatsAndDogsDataModule
from data.kaggle.tb_xrays import TBDataModule
from utils.wandb.binary_classification import make_validation_epoch_end


In [8]:
CustomModel.validation_epoch_end=make_validation_epoch_end(pos_label='Tuberculosis',neg_label='Healthy')

In [9]:
def run_with_mod(num_extras, name=None):
    #pl.seed_everything(42)
    name = '{}_{}'.format(num_extras, '_'.join(names.get_full_name().split()))
    wandb_logger = WandbLogger(name=name,
                               project='Classification_With_Uncertainity')

    model = CustomModel()
    dm = modify_data_module(TBDataModule(),num_extras)
    dm.prepare_data()
    dm.setup()

    model.steps_per_epoch = len(dm.train_dataloader())

    freeze_max_epochs = 1
    unfreeze_max_epochs = 3

    model.epochs = freeze_max_epochs

    model.freeze()

    trainer = pl.Trainer(
        gpus=1,
        max_epochs=freeze_max_epochs,
        logger=wandb_logger
    )
    trainer.fit(model, dm)
    global_step = trainer.global_step
    current_epoch = trainer.current_epoch

    model.epochs=unfreeze_max_epochs
    model.unfreeze()

    trainer = pl.Trainer(
        gpus=1,
        max_epochs=freeze_max_epochs + unfreeze_max_epochs,
        logger=wandb_logger
    )
    trainer.current_epoch = current_epoch + 1
    trainer.global_step = global_step + 1
    trainer.fit(model, dm)

    wandb.finish()


In [ ]:
import os

In [ ]:
for num_extras in (0,1,2,10):
  run_with_mod(num_extras)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: Currently logged in as: borundev (use `wandb login --relogin` to force relogin)



  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 21.8 M
-------------------------------------
528 K     Trainable params
21.3 M    Non-trainable params
21.8 M    Total params
87.252    Total estimated model params size (MB)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 21.8 M
-------------------------------------
21.8 M    Trainable params
0         Non-trainable params
21.8 M    Total params
87.252    Total estimated model params size (MB)
